## Text Classification

**BentoML makes moving trained ML models to production easy:**

* Package models trained with **any ML framework** and reproduce them for model serving in production
* **Deploy anywhere** for online API serving or offline batch serving
* High-Performance API model server with *adaptive micro-batching* support
* Central hub for managing models and deployment process via Web UI and APIs
* Modular and flexible design making it *adaptable to your infrastrcuture*

BentoML is a framework for serving, managing, and deploying machine learning models. It is aiming to bridge the gap between Data Science and DevOps, and enable teams to deliver prediction services in a fast, repeatable, and scalable way.

Before reading this example project, be sure to check out the [Getting started guide](https://github.com/bentoml/BentoML/blob/master/guides/quick-start/bentoml-quick-start-guide.ipynb) to learn about the basic concepts in BentoML.


FastText is a library for efficient learning of word representations and sentence classification.

The goal of text classification is to assign documents (such as emails, posts, text messages, product reviews, etc...) to one or multiple categories. Such categories can be review scores, spam v.s. non-spam, or the language in which the document was typed. Nowadays, the dominant approach to build such classifiers is machine learning, that is learning classification rules from examples. In order to build such classifiers, we need labeled data, which consists of documents and their corresponding categories (or tags, or labels).

As an example, it builds a classifier which automatically classifies stackexchange questions about cooking into one of several possible tags, such as `pot`, `bowl` or `baking`.

This example notebook is base on the guide from fasttext: https://fasttext.cc/docs/en/supervised-tutorial.html

![Impression](https://www.google-analytics.com/collect?v=1&tid=UA-112879361-3&cid=555&t=event&ec=fasttext&ea=fasttext-text-classification&dt=fasttext-text-classification)

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [3]:
!pip install -q bentoml "fasttext==0.9.2"

### Prepare data

In [2]:
!curl https://dl.fbaipublicfiles.com/fasttext/data/cooking.stackexchange.tar.gz --output cooking.stackexchange.tar.gz && tar xvzf cooking.stackexchange.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  446k  100  446k    0     0   466k      0 --:--:-- --:--:-- --:--:--  465k
x cooking.stackexchange.id
x cooking.stackexchange.txt
x readme.txt


In [3]:
!head cooking.stackexchange.txt

__label__sauce __label__cheese How much does potato starch affect a cheese sauce recipe?
__label__food-safety __label__acidity Dangerous pathogens capable of growing in acidic environments
__label__cast-iron __label__stove How do I cover up the white spots on my cast iron stove?
__label__restaurant Michelin Three Star Restaurant; but if the chef is not there
__label__knife-skills __label__dicing Without knife skills, how can I quickly and accurately dice vegetables?
__label__storage-method __label__equipment __label__bread What's the purpose of a bread box?
__label__baking __label__food-safety __label__substitutions __label__peanuts how to seperate peanut oil from roasted peanuts at home?
__label__chocolate American equivalent for British chocolate terms
__label__baking __label__oven __label__convection Fan bake vs bake
__label__sauce __label__storage-lifetime __label__acidity __label__mayonnaise Regulation and balancing of readymade packed mayonnaise and other sauces


In [4]:
!head -n 12404 cooking.stackexchange.txt > cooking.train
!tail -n 3000 cooking.stackexchange.txt > cooking.valid

### Train model

In [5]:
import fasttext

In [6]:
model = fasttext.train_supervised(input="cooking.train")

### Define and save BentoService

In [7]:
%%writefile text_classification.py

from bentoml import env, artifacts, BentoService, api
from bentoml.frameworks.fasttext import FasttextModelArtifact
from bentoml.adapters import JsonInput

@env(infer_pip_packages=True)
@artifacts([FasttextModelArtifact('model')])
class FasttextClassification(BentoService):
    
    @api(input=JsonInput(), batch=True)
    def predict(self, json_list):
        input = [i['text'] for i in json_list]
        result = self.artifacts.model.predict(input)
        # return top result
        prediction_result = [i[0].replace('__label__', '') for i in result[0]]
        return prediction_result

Writing text_classification.py


In [8]:
from text_classification import FasttextClassification

svc = FasttextClassification()
svc.pack('model', model)

saved_path = svc.save()

[2020-09-22 12:09:52,940] WARNING - Using BentoML installed in `editable` model, the local BentoML repository including all code changes will be packaged together with saved bundle created, under the './bundled_pip_dependencies' directory of the saved bundle.
[2020-09-22 12:09:53,839] INFO - Using default docker base image: `None` specified inBentoML config file or env var. User must make sure that the docker base image either has Python 3.7 or conda installed.
[2020-09-22 12:09:55,087] INFO - Detected non-PyPI-released BentoML installed, copying local BentoML modulefiles to target saved bundle path..


no previously-included directories found matching 'e2e_tests'
no previously-included directories found matching 'tests'
no previously-included directories found matching 'benchmark'


UPDATING BentoML-0.9.0rc0+3.gcebf2015/bentoml/_version.py
set BentoML-0.9.0rc0+3.gcebf2015/bentoml/_version.py to '0.9.0.pre+3.gcebf2015'
[2020-09-22 12:10:00,091] INFO - BentoService bundle 'FasttextClassification:20200922120954_28C4D0' saved to: /Users/bozhaoyu/bentoml/repository/FasttextClassification/20200922120954_28C4D0


## REST API Model Serving


To start a REST API model server with the BentoService saved above, use the bentoml serve command:

In [12]:
!bentoml serve {saved_path}

[2020-09-22 12:11:58,505] INFO - Starting BentoML API server in development mode..
[2020-09-22 12:11:58,967] WARNING - Using BentoML installed in `editable` model, the local BentoML repository including all code changes will be packaged together with saved bundle created, under the './bundled_pip_dependencies' directory of the saved bundle.
[2020-09-22 12:11:58,981] WARNING - Saved BentoService bundle version mismatch: loading BentoService bundle create with BentoML version 0.9.0.pre, but loading from BentoML version 0.9.0.pre+3.gcebf2015
[2020-09-22 12:11:59,380] INFO - Using default docker base image: `None` specified inBentoML config file or env var. User must make sure that the docker base image either has Python 3.7 or conda installed.
 * Serving Flask app "FasttextClassification" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
[2020-09-22 12:12:17,344] INFO - {'servic

If you are running this notebook from Google Colab, you can start the dev server with `--run-with-ngrok` option, to gain acccess to the API endpoint via a public endpoint managed by [ngrok](https://ngrok.com/):

In [11]:
!bentoml serve FasttextClassification:latest --run-with-ngrok

[2020-09-22 12:11:20,296] INFO - Getting latest version FasttextClassification:20200922120954_28C4D0
[2020-09-22 12:11:20,297] INFO - Starting BentoML API server in development mode..
[2020-09-22 12:11:20,550] WARNING - Using BentoML installed in `editable` model, the local BentoML repository including all code changes will be packaged together with saved bundle created, under the './bundled_pip_dependencies' directory of the saved bundle.
[2020-09-22 12:11:20,564] WARNING - Saved BentoService bundle version mismatch: loading BentoService bundle create with BentoML version 0.9.0.pre, but loading from BentoML version 0.9.0.pre+3.gcebf2015
[2020-09-22 12:11:20,902] INFO - Using default docker base image: `None` specified inBentoML config file or env var. User must make sure that the docker base image either has Python 3.7 or conda installed.
 * Serving Flask app "FasttextClassification" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


Open http://127.0.0.1:5000 to see more information about the REST APIs server in your
browser.


### Send prediction requeset to the REST API server

Navigate to parent directory of the notebook(so you have reference to the `test.jpg` image), and run the following `curl` command to send the image to REST API server and get a prediction result:

```
curl -X POST \
  http://localhost:5000/predict \
  -H 'Content-Type: application/json' \
  -d '{"text": "Which baking dish is best to bake a banana bread ?"}'
```

## Containerize model server with Docker


One common way of distributing this model API server for production deployment, is via Docker containers. And BentoML provides a convenient way to do that.

Note that docker is **not available in Google Colab**. You will need to download and run this notebook locally to try out this containerization with docker feature.

If you already have docker configured, simply run the follow command to product a docker container serving the IrisClassifier prediction service created above:

In [13]:
!bentoml containerize FasttextClassification:latest

[2020-09-22 12:13:07,456] INFO - Getting latest version FasttextClassification:20200922120954_28C4D0
Found Bento: /Users/bozhaoyu/bentoml/repository/FasttextClassification/20200922120954_28C4D0
[2020-09-22 12:13:07,495] WARNING - Using BentoML installed in `editable` model, the local BentoML repository including all code changes will be packaged together with saved bundle created, under the './bundled_pip_dependencies' directory of the saved bundle.
[2020-09-22 12:13:07,511] WARNING - Saved BentoService bundle version mismatch: loading BentoService bundle create with BentoML version 0.9.0.pre, but loading from BentoML version 0.9.0.pre+3.gcebf2015
Tag not specified, using tag parsed from BentoService: 'fasttextclassification:20200922120954_28C4D0'
Building Docker image fasttextclassification:20200922120954_28C4D0 from FasttextClassification:latest 
-we in here
processed docker file
(None, None)
root in create archive /Users/bozhaoyu/bentoml/repository/FasttextClassification/20200922120

/Requirement already satisfied: docker in /opt/conda/lib/python3.7/site-packages (from bentoml==0.9.0.pre->-r ./requirements.txt (line 1)) (4.3.1)
|Requirement already satisfied: cerberus in /opt/conda/lib/python3.7/site-packages (from bentoml==0.9.0.pre->-r ./requirements.txt (line 1)) (1.3.2)
\Requirement already satisfied: chardet<4.0,>=2.0 in /opt/conda/lib/python3.7/site-packages (from aiohttp->bentoml==0.9.0.pre->-r ./requirements.txt (line 1)) (3.0.4)
-Requirement already satisfied: typing-extensions>=3.7.4; python_version < "3.8" in /opt/conda/lib/python3.7/site-packages (from yarl<2.0,>=1.0->aiohttp->bentoml==0.9.0.pre->-r ./requirements.txt (line 1)) (3.7.4.3)
-  Building wheel for fasttext (setup.py): finished with status 'done'
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=4441002 sha256=c9a69b2133d5b1c05b68f9be59d9e74583689c299224445f20de0dcfc227432a
  Stored in directory: /tmp/pip-ephem-wheel-cache-sx8v7vyg/wheels/4e/ca/bf/b02

\Successfully installed fasttext-0.9.2
\ ---> a90f620bbe4d
Step 8/15 : COPY . /bento
\ ---> 5d2abfb72a02
Step 9/15 : RUN if [ -d /bento/bundled_pip_dependencies ]; then pip install -U bundled_pip_dependencies/* ;fi
 ---> Running in 1f2cf07c7029
|Processing ./bundled_pip_dependencies/BentoML-0.9.0rc0+3.gcebf2015.tar.gz
|  Installing build dependencies: started
\  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
-  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
|    Preparing wheel metadata: finished with status 'done'
\Requirement already satisfied, skipping upgrade: numpy in /opt/conda/lib/python3.7/site-packages (from BentoML==0.9.0rc0+3.gcebf2015) (1.19.2)
-Requirement already satisfied, skipping upgrade: sqlalchemy-utils<0.36.8 in /opt/conda/lib/python3.7/site-packages (from BentoML==0.9.0rc0+3.gcebf2015) (0.36.7)
/Requirement already satisfied, skipping upgrad

/  Building wheel for BentoML (PEP 517): finished with status 'done'
  Created wheel for BentoML: filename=BentoML-0.9.0rc0+3.gcebf2015-py3-none-any.whl size=3064091 sha256=0bd361a45c83e2bb80aa10347199c1fd71684e3a1389eec1208f54b4297c1140
  Stored in directory: /root/.cache/pip/wheels/a0/45/41/62152db705af4ff47e7a3d6abf6247986eef4aa1b94a58d3b9
Successfully built BentoML
/Installing collected packages: BentoML
  Attempting uninstall: BentoML
    Found existing installation: BentoML 0.9.0rc0
\    Uninstalling BentoML-0.9.0rc0:
/      Successfully uninstalled BentoML-0.9.0rc0
\Successfully installed BentoML-0.9.0rc0+3.gcebf2015
/ ---> 860b9bec3b35
Step 10/15 : ENV PORT 5000
| ---> Running in c2743985d5de
\ ---> c3f39fdc8450
Step 11/15 : EXPOSE $PORT
 ---> Running in 0733c7dcfea4
- ---> bd7997accbc4
Step 12/15 : COPY docker-entrypoint.sh /usr/local/bin/
| ---> 71c57d9f3c42
Step 13/15 : RUN chmod +x /usr/local/bin/docker-entrypoint.sh
\ ---> Running in 8e7c893f2534
/ ---> e1c5868

In [16]:
!docker run --rm -p5000:5000 fasttextclassification:20200922120954_28C4D0

[2020-09-22 19:17:58,674] INFO - Starting BentoML API server in production mode..
[2020-09-22 19:17:59,100] INFO - get_gunicorn_num_of_workers: 3, calculated by cpu count
[2020-09-22 19:17:59 +0000] [1] [INFO] Starting gunicorn 20.0.4
[2020-09-22 19:17:59 +0000] [1] [INFO] Listening at: http://0.0.0.0:5000 (1)
[2020-09-22 19:17:59 +0000] [1] [INFO] Using worker: sync
[2020-09-22 19:17:59 +0000] [11] [INFO] Booting worker with pid: 11
[2020-09-22 19:17:59 +0000] [12] [INFO] Booting worker with pid: 12
[2020-09-22 19:17:59 +0000] [13] [INFO] Booting worker with pid: 13
[2020-09-22 19:17:59,365] WARNING - Using BentoML not from official PyPI release. In order to find the same version of BentoML when deploying your BentoService, you must set the 'core/bentoml_deploy_version' config to a http/git location of your BentoML fork, e.g.: 'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-09-22 19:17:59,387] WARNING - Saved BentoService bundle version mismatch

## Load saved BentoService

bentoml.load is the API for loading a BentoML packaged model in python:

In [17]:
from bentoml import load

svc = load(saved_path)

print(svc.predict([{"text": "which baking dish is the best?"}]))

[2020-09-22 12:18:22,256] WARNING - Saved BentoService bundle version mismatch: loading BentoService bundle create with BentoML version 0.9.0.pre, but loading from BentoML version 0.9.0.pre+3.gcebf2015
[2020-09-22 12:18:22,257] WARNING - Module `text_classification` already loaded, using existing imported module.
[2020-09-22 12:18:22,278] WARNING - pip package requirement fasttext already exist
['baking']


## Launch inference job from CLI

BentoML cli supports loading and running a packaged model from CLI. With the DataframeInput adapter, the CLI command supports reading input Dataframe data from CLI argument or local csv or json files:

In [18]:
!bentoml run FasttextClassification:latest predict --input='{"text": "Which baking dish is best to bake a banana bread ?"}'

[2020-09-22 12:18:42,572] INFO - Getting latest version FasttextClassification:20200922120954_28C4D0
[2020-09-22 12:18:42,609] WARNING - Using BentoML installed in `editable` model, the local BentoML repository including all code changes will be packaged together with saved bundle created, under the './bundled_pip_dependencies' directory of the saved bundle.
[2020-09-22 12:18:42,623] WARNING - Saved BentoService bundle version mismatch: loading BentoService bundle create with BentoML version 0.9.0.pre, but loading from BentoML version 0.9.0.pre+3.gcebf2015
[2020-09-22 12:18:43,064] INFO - Using default docker base image: `None` specified inBentoML config file or env var. User must make sure that the docker base image either has Python 3.7 or conda installed.
[2020-09-22 12:18:47,361] INFO - {'service_name': 'FasttextClassification', 'service_version': '20200922120954_28C4D0', 'api': 'predict', 'task': {'data': '{"text": "Which baking dish is best to bake a banana bread ?"}', 'task_id':

# Deployment Options

If you are at a small team with limited engineering or DevOps resources, try out automated deployment with BentoML CLI, currently supporting AWS Lambda, AWS SageMaker, and Azure Functions:
- [AWS Lambda Deployment Guide](https://docs.bentoml.org/en/latest/deployment/aws_lambda.html)
- [AWS SageMaker Deployment Guide](https://docs.bentoml.org/en/latest/deployment/aws_sagemaker.html)
- [Azure Functions Deployment Guide](https://docs.bentoml.org/en/latest/deployment/azure_functions.html)

If the cloud platform you are working with is not on the list above, try out these step-by-step guide on manually deploying BentoML packaged model to cloud platforms:
- [AWS ECS Deployment](https://docs.bentoml.org/en/latest/deployment/aws_ecs.html)
- [Google Cloud Run Deployment](https://docs.bentoml.org/en/latest/deployment/google_cloud_run.html)
- [Azure container instance Deployment](https://docs.bentoml.org/en/latest/deployment/azure_container_instance.html)
- [Heroku Deployment](https://docs.bentoml.org/en/latest/deployment/heroku.html)

Lastly, if you have a DevOps or ML Engineering team who's operating a Kubernetes or OpenShift cluster, use the following guides as references for implementating your deployment strategy:
- [Kubernetes Deployment](https://docs.bentoml.org/en/latest/deployment/kubernetes.html)
- [Knative Deployment](https://docs.bentoml.org/en/latest/deployment/knative.html)
- [Kubeflow Deployment](https://docs.bentoml.org/en/latest/deployment/kubeflow.html)
- [KFServing Deployment](https://docs.bentoml.org/en/latest/deployment/kfserving.html)
- [Clipper.ai Deployment Guide](https://docs.bentoml.org/en/latest/deployment/clipper.html)

